# Calcium Data Visualization Code
This code is to read in calcium imaging traces from a .mat file and visualize the responses of the different traces.

This data is saved in BME-HAN-STORAGE/Share/People/Hua-an/Data/TBI/Ali/

In [ ]:
"""
Requires These Packages for Functions to Run Functions
"""
import numpy as np
import thunder as td
import glob
import scipy.io as sio
import scipy.stats as sstats
from __future__ import division

"""
These Packages are only required for display/working in this specific notebook
"""
%matplotlib inline

import matplotlib.pyplot as plt
from showit import image

In [ ]:
#PDF Vectorization for Copying Figures
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 300

#plt.rcParams['figure.autolayout'] = False
#plt.rcParams['figure.figsize'] = 10, 6
#plt.rcParams['axes.labelsize'] = 8
#plt.rcParams['axes.titlesize'] = 20
#plt.rcParams['font.size'] = 8
#plt.rcParams['lines.linewidth'] = 2.0
#plt.rcParams['lines.markersize'] = 8
#plt.rcParams['legend.fontsize'] = 14

#plt.rcParams['text.usetex'] = True
#plt.rcParams['font.family'] = "serif"
#plt.rcParams['font.serif'] = "cm"
#plt.rcParams['text.latex.preamble'] = "\usepackage{subdepth}, \usepackage{type1cm}"

In [ ]:
# Locate Data
basedir = '/mnt/eng_research_handata/Hua-an/Data/TBI/Ali/'
beforedir = basedir + 'TBI_BEFORE BLASTING DATA ANALYSIS/'
afterdir = basedir + 'TBI_AFTER BLASTING DATA ANALYSIS/'
beforefiles = sorted(glob.glob(beforedir+'**/**/R_bins_circle_20170804.mat')) #R.mat
beforefiles.sort(key=len)
afterfiles = sorted(glob.glob(afterdir+'**/**/R_bins_circle_20170804.mat')) #R.mat
afterfiles.sort(key=len)
#Ali 6,9,12 and Seth 5,6 (13,14) are EXP Mice.  Ali 8,10,11 and Moona 1,2 (15,16) are CTL Mice
celldict = {6: ((0,1),(4,5)), 8: ((2,3),(14,15)), 9: ((4,5),(6,7)), 10: ((6,7),(16,17)),
            11: ((8,9),(18,19)), 12: ((10,11),(8,9)), 13: ((12,13),(10,11)), 
            14: ((14,15),(12,13)), 15: ((16,17),(0,1)), 16: ((18,19),(2,3))}
beforefiles, afterfiles

In [ ]:
range(len(beforefiles))

In [ ]:
# Code for Before files Analysis and Output
beforetraces = range(len(beforefiles))
beforekeeptraces = range(len(beforefiles))
beforekeepbinary = range(len(beforefiles))
sample_time = .05 # 50 ms for 20Hz imaging

for idx, mfile in enumerate(beforefiles):
    # Load mat file
    openmat = sio.loadmat(mfile, squeeze_me=True, struct_as_record=False)
    fid = mfile[-22:] #Get last 22 characters of filename to save as identifier
    r_out = openmat['r_out']
    NCells = r_out.shape[0]
    trlength = r_out[0].trace.shape[0]
    #file_struct = np.copy(r_out['file'])
    #traces = np.copy(file_struct['trace'])
    #fnorm_traces = np.copy(file_struct['fnormtrace'])
    #df_traces = np.copy(file_struct['dftrace'])
    #stdKeepTraces = np.copy(r_out['stdKeepTrace'])
    #peakMatrix = np.copy(r_out['peakMatrix'])
    # Create Array of all traces from Input Data
    beforetime = np.arange(0, trlength*sample_time, sample_time)
    beforetraces[idx] = np.zeros((NCells, trlength))
    beforekeeptraces[idx] = np.zeros((1,trlength))
    beforekeepbinary[idx] = np.zeros((1,trlength))
    for step, struct in enumerate(r_out[:-1]): #-1 to ignore last ROI which should be background/blank ROI
        beforetraces[idx][step,:] = struct.file.dftrace #dftrace #df for fig4.  fnorm for fig2.
    #count=0
    #for ind, bintrace in enumerate(peakMatrix):
    #    if np.sum(~np.isnan(bintrace)):
    #        if np.equal(count,0):
    #            beforekeepbinary[idx][count,:] = bintrace
    #        else:
    #            beforekeepbinary[idx] = np.concatenate((beforekeepbinary[idx],bintrace.reshape(1,-1)),axis=0)
    #        count += 1
    #count=0
    #for ind, filttrace in enumerate(stdKeepTraces):
    #    if np.sum(~np.isnan(filttrace)):
    #        if np.equal(count,0):
    #            beforekeeptraces[idx][count,:] = filttrace
    #        else:
    #            beforekeeptraces[idx] = np.concatenate((beforekeeptraces[idx],filttrace.reshape(1,-1)),axis=0)
    #        count += 1
        
allbeforetraces = np.vstack(beforetraces)
#allbeforekeeptraces = np.vstack(beforekeeptraces)
#allbeforekeepbinary = np.vstack(beforekeepbinary)
allbeforetraces.shape#, allbeforekeeptraces.shape, allbeforekeepbinary.shape

In [ ]:
# Code for After files Analysis and Output
aftertraces = range(len(afterfiles))
traces = range(len(afterfiles))
afterkeeptraces = range(len(afterfiles))
keeptraces = range(len(afterfiles))
afterkeepbinary = range(len(afterfiles))
keepbinary = range(len(afterfiles))

sample_time = .05

for idx, mfile in enumerate(afterfiles):
    # Load mat file
    openmat = sio.loadmat(mfile, squeeze_me=True, struct_as_record=False)
    fid = mfile[-21:] #Get last 22 characters of filename to save as identifier
    r_out = openmat['r_out']
    NCells = r_out.shape[0]
    trlength = r_out[0].file[0].trace.shape[0]
    catfiles = r_out[0].file.shape[0]
    #df_traces = np.copy(r_out['Trace'])
    #stdKeepTraces = np.copy(r_out['stdKeepTrace'])
    #peakMatrix = np.copy(r_out['peakMatrix'])
    # Create Array of all traces from Input Data
    aftertraces[idx] = np.zeros((catfiles,NCells,trlength))
    for ind, struct in enumerate(r_out[:-1]): #-1 to ignore last ROI which should be background/blank
        for step, fstruct in enumerate(struct.file):
            aftertraces[idx][step,ind,:] = fstruct.dftrace #dftrace #df for fig4.  fnorm for fig2.
    
    #aftertime = np.arange(0, trlength*sample_time, sample_time)
    #traces[idx] = np.zeros((NCells, trlength))
    #keeptraces[idx] = np.zeros((1, trlength))
    #keepbinary[idx] = np.zeros((1, trlength))
    #for step, trace in enumerate(df_traces):
    #    traces[idx][step,:] = trace
    #count = 0
    #for ind, bintrace in enumerate(peakMatrix):
    #    if np.sum(~np.isnan(bintrace)):
    #        if np.equal(count,0):
    #            keepbinary[idx][count,:] = bintrace
    #        else:
    #            keepbinary[idx] = np.concatenate((keepbinary[idx],bintrace.reshape(1,-1)),axis=0)
    #        count += 1
    #count=0
    #for ind, filttrace in enumerate(stdKeepTraces):
    #    if np.sum(~np.isnan(filttrace)):
    #        if np.equal(count,0):
    #            keeptraces[idx][count,:] = filttrace
    #        else:
    #            keeptraces[idx] = np.concatenate((keeptraces[idx],filttrace.reshape(1,-1)),axis=0)
    #        count += 1
    # Divide concatenated after data into individual trials
    #num_cells = traces[idx].shape[0]
    #keep_cells = keeptraces[idx].shape[0]
    #flength = traces[idx].shape[1]/catfiles
    #start, stop, step = 0, flength, 0
    #aftertraces[idx] = np.zeros((catfiles,num_cells,flength))
    #afterkeeptraces[idx] = np.zeros((catfiles,keep_cells,flength))
    #afterkeepbinary[idx] = np.zeros((catfiles,keep_cells,flength))
    #while step < catfiles:
    #    # Sample Beginning and End Timepoints
    #    aftertraces[idx][step,:,:] = np.copy(traces[idx][:,start:stop])
    #    afterkeeptraces[idx][step,:,:] = np.copy(keeptraces[idx][:,start:stop])
    #    afterkeepbinary[idx][step,:,:] = np.copy(keepbinary[idx][:,start:stop])
    #    start += flength
    #    stop += flength
    #    step += 1

allaftertraces = np.concatenate(aftertraces, axis=1)
#allafterkeeptraces = np.concatenate(afterkeeptraces, axis=1)
#allafterkeepbinary = np.concatenate(afterkeepbinary, axis=1)
allaftertraces.shape#, allafterkeeptraces.shape, allafterkeepbinary.shape

In [ ]:
allaftertraces.shape, aftertraces[10].shape

In [ ]:
beforeexpcells = list(celldict[6][0])
beforeexpcells.extend(celldict[9][0])
beforeexpcells.extend(celldict[12][0])
beforeexpcells.extend(celldict[13][0])
beforeexpcells.extend(celldict[14][0])
afterexpcells = list(celldict[6][1])
afterexpcells.extend(celldict[9][1])
afterexpcells.extend(celldict[12][1])
afterexpcells.extend(celldict[13][1])
afterexpcells.extend(celldict[14][1])

beforectlcells = list(celldict[8][0])
beforectlcells.extend(celldict[10][0])
beforectlcells.extend(celldict[11][0])
beforectlcells.extend(celldict[15][0])
beforectlcells.extend(celldict[16][0])
afterctlcells = list(celldict[8][1])
afterctlcells.extend(celldict[10][1])
afterctlcells.extend(celldict[11][1])
afterctlcells.extend(celldict[15][1])
afterctlcells.extend(celldict[16][1])

#Pick specific animals to look at
before6d1 = beforetraces[list(celldict[6][0])[0]]
after6d1 = aftertraces[list(celldict[6][1])[0]]

#before6bin = beforebinary[list(celldict[6][0])[0]]
#after6bin = afterbinary[list(celldict[6][0])[0]]

before11d1 = beforetraces[list(celldict[11][0])[0]]
after11d1 = aftertraces[list(celldict[11][1])[0]]

#Create Population Sets for Data
expbeforetraces = np.vstack(beforetraces[idx] for idx in beforeexpcells)
ctlbeforetraces = np.vstack(beforetraces[idx] for idx in beforectlcells)
#Binary Traces
#expbeforebinary = np.vstack(beforekeepbinary[idx] for idx in beforeexpcells)
#ctlbeforebinary = np.vstack(beforekeepbinary[idx] for idx in beforectlcells)

expaftertraces = np.concatenate(list(aftertraces[idx] for idx in afterexpcells), axis=1)
ctlaftertraces = np.concatenate(list(aftertraces[idx] for idx in afterctlcells), axis=1)
#Binary Traces
#expafterbinary = np.concatenate(list(afterkeepbinary[idx] for idx in afterexpcells), axis=1)
#ctlafterbinary = np.concatenate(list(afterkeepbinary[idx] for idx in afterctlcells), axis=1)

In [ ]:
#Determine Time Trace
numframes = allbeforetraces.shape[1]
imgrate = 20 #Image rate in Hz
endtime = numframes/imgrate
tickstep = 500
allticks = np.arange(0, numframes, tickstep)
fulltracetime = np.arange(0, endtime, 1/imgrate)
timelabels = np.arange(0, endtime, tickstep/imgrate, dtype=np.int)
#timelabels = np.insert(tracetime,0,0)
allticks, timelabels, fulltracetime.shape

In [ ]:
after6d1.shape

In [ ]:
#Single Experimental Mouse (6 Day 1)
meanbefore = before6d1.mean(axis=1)
sortindsbefore = np.argsort(meanbefore)[::-1]
meanfirstafter = after6d1[0,:,:].mean(axis=1)
sortindsafter = np.argsort(meanfirstafter)[::-1]

labelsize = 12
axsize = 9
cbarsize = 10

fig, all_ax = plt.subplots(nrows=1,ncols=6, figsize=(11,3))

im=range(6)
for idx, ax in enumerate(all_ax):
    if idx==0: #-100 from traces for fnorm traces to be centered at 0
        im[idx] = ax.imshow((100*(before6d1[sortindsbefore,:])), cmap='seismic')
        ax.set_aspect(7, adjustable='box')
        im[idx].set_clim(-20,20) #dftraces [-20, 20] fnormtraces
        ax.set_ylabel('Cell Number', fontsize=labelsize)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticks(allticks)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        plt.setp(ax.get_yticklabels(), fontsize=axsize)
    elif idx==1: 
        im[idx] = ax.imshow((100*(after6d1[idx-1,sortindsafter,:])), cmap='seismic')
        ax.set_aspect(9)
        im[idx].set_clim(-20,20)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticks(allticks)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        plt.setp(ax.get_yticklabels(), fontsize=axsize)
    else: 
        im[idx] = ax.imshow((100*(after6d1[idx-1,sortindsafter,:])), cmap='seismic')
        ax.set_aspect(9)
        im[idx].set_clim(-20,20)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticks(allticks)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticklabels([])
        
# Add Colorbar
fig.subplots_adjust(right=0.95, wspace=0.3)
cbar_ax = fig.add_axes([0.985, 0.12, 0.02, 0.75])
fig.colorbar(im[idx], cax=cbar_ax)
cbar_ax.tick_params(labelsize=cbarsize)

# Add Title Info
#fig.suptitle('Experimental Mouse Sorted by Mean Post-Blast', fontsize=14, fontweight='bold')

In [ ]:
laststds = after6d1[4,sortindsafter,:].std(axis=1)
firstmeans = after6d1[0,sortindsafter,:].mean(axis=1)
plt.plot(laststds)
plt.plot(firstmeans)
updiff = firstmeans >= 3*laststds #62 if mean of stds, 82 if std cell by cell
downdiff = -1*firstmeans >= 3*laststds #6 if mean of stds, 21 if std cell by cell
updiff.sum(), downdiff.sum(), firstmeans.shape, laststds.mean(), before6d1.shape, after6d1.shape

In [ ]:
sortindsafter[updiff], sortindsafter[downdiff]

In [ ]:
beforesort = before6d1[sortindsbefore,:]
aftersort = after6d1[:,sortindsafter,:]

#Cell Choices
beforechoice = np.array([0, 100, 203, 303, 401, 501])
choice = np.array([5, 99, 200, 300, 400, 460])
numchoice = len(choice)
#Time Choices
timeperiods = np.array([0,0,2,4]) #0-for beforesort, 0-first in aftersort, 2-mid aftersort, 4-last aftersort
#Max Y-Axes Choices
maxaxes = np.array([50, 50, 50, 50])
#Text Values
labelsize = 12
axsize = 9
cbarsize = 10
lwidth = 3

for step, time in enumerate(timeperiods):
    if (step, time) == (0, 0):
        clrs = np.array([[0.87,0.87,0.14],[1,0.34,0.34],[0,1,0],[0.57,0.84,0.98],[0.5,0,1],[1,0.52,0.74]])
    else:
        clrs = np.array([[1,0,0],[0.16,0.5,0],[1,0,1],[0,0,1],[0,0,0],[1,0.62,0]])
    
    fig, all_ax = plt.subplots(nrows=numchoice,ncols=1, figsize=(12,6))

    tr=range(numchoice)
    for idx, ax in enumerate(all_ax):
        if step==0:
            tr[idx] = ax.plot(fulltracetime, (100*beforesort[beforechoice[idx],:]), linewidth=lwidth, c=clrs[idx])
            ax.set_ylim([-50,maxaxes[step]])
            ax.set_ylabel('dF/F', fontsize=labelsize)
            ax.set_xlabel('Time (sec)', fontsize=labelsize)
            #ax.set_xticklabels(tracetime, fontsize=axsize)
            plt.setp(ax.get_yticklabels(), fontsize=axsize)
        else: 
            tr[idx] = ax.plot(fulltracetime, (100*aftersort[time,choice[idx],:]), linewidth=lwidth, c=clrs[idx])
            ax.set_ylim([-50,maxaxes[step]])
            ax.set_ylabel('dF/F', fontsize=labelsize)
            ax.set_xlabel('Time (sec)', fontsize=labelsize)
            #ax.set_xticklabels(tracetime, fontsize=axsize)
            plt.setp(ax.get_yticklabels(), fontsize=axsize)


In [ ]:
#Single Control Mouse (11 Day 1)
meanbefore = before11d1.mean(axis=1)
sortindsbefore = np.argsort(meanbefore)[::-1]
meanfirstafter = after11d1[0,:,:].mean(axis=1)
sortindsafter = np.argsort(meanfirstafter)[::-1]

labelsize = 12
axsize = 9
cbarsize = 10

fig, all_ax = plt.subplots(nrows=1,ncols=6, figsize=(11,3))

im=range(6)
for idx, ax in enumerate(all_ax):
    if idx==0: #-100 from each trace for fnormtraces
        im[idx] = ax.imshow((100*(before11d1[sortindsbefore,:])), cmap='seismic')
        ax.set_aspect(7, adjustable='box')
        im[idx].set_clim(-20,20)
        ax.set_ylabel('Cell Number', fontsize=labelsize)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticks(allticks)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        plt.setp(ax.get_yticklabels(), fontsize=axsize)
    elif idx==1: 
        im[idx] = ax.imshow((100*(after11d1[idx-1,sortindsafter,:])), cmap='seismic')
        ax.set_aspect(5)
        im[idx].set_clim(-20,20)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticks(allticks)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        plt.setp(ax.get_yticklabels(), fontsize=axsize)
    else: 
        im[idx] = ax.imshow((100*(after11d1[idx-1,sortindsafter,:])), cmap='seismic')
        ax.set_aspect(5)
        im[idx].set_clim(-20,20)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticks(allticks)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticklabels([])
        
# Add Colorbar
fig.subplots_adjust(right=0.95, wspace=0.3)
cbar_ax = fig.add_axes([0.975, 0.12, 0.02, 0.7])
fig.colorbar(im[idx], cax=cbar_ax)
cbar_ax.tick_params(labelsize=cbarsize)

# Add Title Info
#fig.suptitle('Experimental Mouse Sorted by Mean Post-Blast', fontsize=14, fontweight='bold')

In [ ]:
laststds = after11d1[4,sortindsafter,:].std(axis=1)
firstmeans = after11d1[0,sortindsafter,:].mean(axis=1)
plt.plot(laststds)
plt.plot(firstmeans)
updiff = firstmeans >= 3*laststds #1 if mean, 24 if cell by cell (std<5 roughly)
downdiff = -1*firstmeans >= 3*laststds #0 if mean, 0 if cell by cell
updiff.sum(), downdiff.sum(), firstmeans.shape, laststds.mean()

In [ ]:
beforesort = before11d1[sortindsbefore,:]
aftersort = after11d1[:,sortindsafter,:]

#Cell Choices
beforechoice = np.array([1, 100, 251, 380, 500, 601])
choice = np.array([10, 150, 302, 451, 601, 750])
numchoice = len(choice)
#Time Choices
timeperiods = np.array([0,0,2,4]) #0-for beforesort, 0-first in aftersort, 2-mid aftersort, 4-last aftersort
#Max Y-Axes Choices
maxaxes = np.array([50, 50, 50, 50])
#Text Values
labelsize = 12
axsize = 9
cbarsize = 10
lwidth = 3

for step, time in enumerate(timeperiods):
    if (step, time) == (0, 0):
        clrs = np.array([[0.87,0.87,0.14],[1,0.34,0.34],[0,1,0],[0.57,0.84,0.98],[0.5,0,1],[1,0.52,0.74]])
    else:
        clrs = np.array([[1,0,0],[0.16,0.5,0],[1,0,1],[0,0,1],[0,0,0],[1,0.62,0]])
    
    fig, all_ax = plt.subplots(nrows=numchoice,ncols=1, figsize=(12,6))

    tr=range(numchoice)
    for idx, ax in enumerate(all_ax):
        if step==0:
            tr[idx] = ax.plot(fulltracetime, (100*beforesort[beforechoice[idx],:]), linewidth=lwidth, c=clrs[idx])
            ax.set_ylim([-50,maxaxes[step]])
            ax.set_ylabel('dF/F', fontsize=labelsize)
            ax.set_xlabel('Time (sec)', fontsize=labelsize)
            #ax.set_xticklabels(tracetime, fontsize=axsize)
            plt.setp(ax.get_yticklabels(), fontsize=axsize)
        else: 
            tr[idx] = ax.plot(fulltracetime, (100*aftersort[time,choice[idx],:]), linewidth=lwidth, c=clrs[idx])
            ax.set_ylim([-50,maxaxes[step]])
            ax.set_ylabel('dF/F', fontsize=labelsize)
            ax.set_xlabel('Time (sec)', fontsize=labelsize)
            #ax.set_xticklabels(tracetime, fontsize=axsize)
            plt.setp(ax.get_yticklabels(), fontsize=axsize)


In [ ]:
meanbefore = expbeforetraces.mean(axis=1)
sortindsbefore = np.argsort(meanbefore)[::-1]
meanfirstafter = expaftertraces[0,:,:].mean(axis=1)
sortindsafter = np.argsort(meanfirstafter)[::-1]

labelsize = 14
axsize = 12
cbarsize = 12

fig, all_ax = plt.subplots(nrows=1,ncols=6, figsize=(15,5))

im=range(6)
for idx, ax in enumerate(all_ax):
    if idx==0:
        im[idx] = ax.imshow((100*expbeforetraces[sortindsbefore,:])-100, cmap='seismic')
        ax.set_aspect(1.85, adjustable='box')
        im[idx].set_clim(-50,50)
        ax.set_ylabel('Cell Number', fontsize=labelsize)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        plt.setp(ax.get_yticklabels(), fontsize=axsize)
    elif idx==1: 
        im[idx] = ax.imshow((100*expaftertraces[idx-1,sortindsafter,:])-100, cmap='seismic')
        ax.set_aspect(1.85)
        im[idx].set_clim(-50,50)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        plt.setp(ax.get_yticklabels(), fontsize=axsize)
    else: 
        im[idx] = ax.imshow((100*expaftertraces[idx-1,sortindsafter,:])-100, cmap='seismic')
        ax.set_aspect(1.85)
        im[idx].set_clim(-50,50)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticklabels([])
        
# Add Colorbar
fig.subplots_adjust(right=0.95, wspace=0.2)
cbar_ax = fig.add_axes([0.97, 0.115, 0.02, 0.8])
fig.colorbar(im[idx], cax=cbar_ax)
cbar_ax.tick_params(labelsize=cbarsize)

# Add Title Info
#fig.suptitle('Experimental Sorted by Mean Post-Blast', fontsize=14, fontweight='bold')

In [ ]:
laststds = expaftertraces[4,sortindsafter,:].std(axis=1)
firstmeans = expaftertraces[0,sortindsafter,:].mean(axis=1)
plt.plot(laststds)
plt.plot(firstmeans)
updiff = firstmeans >= 3*laststds #319 if mean, 506 if cell by cell
downdiff = -1*firstmeans >= 3*laststds #103 if mean, 229 if cell by cell
updiff.sum(), downdiff.sum(), firstmeans.shape, laststds.mean()

In [ ]:
#Plot Sets of Matches
upcells = expaftertraces[0,sortindsafter[updiff],:]
upbinary = expafterbinary[0,sortindsafter[updiff],:]
upstds = expaftertraces[4,sortindsafter[updiff],:]
upstdbinary = expafterbinary[4,sortindsafter[updiff],:]

downcells = expaftertraces[0,sortindsafter[downdiff],:]
downbinary = expafterbinary[0,sortindsafter[downdiff],:]
downstds = expaftertraces[4,sortindsafter[downdiff],:]
downstdbinary = expafterbinary[4,sortindsafter[downdiff],:]

labelsize = 14
axsize = 12
cbarsize = 12

fig, all_ax = plt.subplots(nrows=1,ncols=4, figsize=(15,5))

im=range(4)
for idx, ax in enumerate(all_ax):
    if idx==0:
        im[idx] = ax.imshow(upcells, cmap='seismic')
        ax.set_aspect(2)
        im[idx].set_clim(-50,50)
        ax.set_ylabel('Cell Number', fontsize=labelsize)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        plt.setp(ax.get_yticklabels(), fontsize=axsize)
    elif idx==1: 
        im[idx] = ax.imshow(upstds, cmap='seismic')
        ax.set_aspect(2)
        im[idx].set_clim(-50,50)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticklabels([])
    elif idx==2: 
        im[idx] = ax.imshow(downcells, cmap='seismic')
        ax.set_aspect(2)
        im[idx].set_clim(-50,50)
        ax.set_ylabel('Cell Number', fontsize=labelsize)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticks([0,100,200])
        ax.set_yticklabels([0,100,200], fontsize=axsize)
    elif idx==3:
        im[idx] = ax.imshow(downstds, cmap='seismic')
        ax.set_aspect(2)
        im[idx].set_clim(-50,50)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticklabels([])
        
# Add Colorbar
fig.subplots_adjust(right=0.95, wspace=0.3)
cbar_ax = fig.add_axes([0.97, 0.35, 0.02, 0.3])
fig.colorbar(im[idx], cax=cbar_ax)
cbar_ax.tick_params(labelsize=cbarsize)

In [ ]:
choice = 1
plt.plot(upbinary[choice])
plt.plot(upcells[choice])
plt.figure()
plt.plot(upbinary.sum(axis=1))
print(upbinary.sum(axis=1).mean())
print(upcells[choice].mean())

In [ ]:
stdbefore = ctlbeforetraces.mean(axis=1)
sortindsbefore = np.argsort(stdbefore)[::-1]
stdfirstafter = ctlaftertraces[0,:,:].mean(axis=1)
sortindsafter = np.argsort(stdfirstafter)[::-1]

labelsize = 14
axsize = 12
cbarsize = 12

fig, all_ax = plt.subplots(nrows=1,ncols=6, figsize=(15,5))

im=range(6)
for idx, ax in enumerate(all_ax):
    if idx==0:
        im[idx] = ax.imshow(ctlbeforetraces[sortindsbefore,:], cmap='seismic')
        ax.set_aspect(2.8)
        im[idx].set_clim(-50,50)
        ax.set_ylabel('Cell Number', fontsize=labelsize)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        plt.setp(ax.get_yticklabels(), fontsize=axsize)
    elif idx==1: 
        im[idx] = ax.imshow(ctlaftertraces[idx-1,sortindsafter,:], cmap='seismic')
        ax.set_aspect(1.85)
        im[idx].set_clim(-50,50)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        plt.setp(ax.get_yticklabels(), fontsize=axsize)
    else:
        im[idx] = ax.imshow(ctlaftertraces[idx-1,sortindsafter,:], cmap='seismic')
        ax.set_aspect(1.85)
        im[idx].set_clim(-50,50)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticklabels([])
    
# Add Colorbar
fig.subplots_adjust(right=0.95, wspace=0.2)
cbar_ax = fig.add_axes([0.97, 0.115, 0.02, 0.8])
fig.colorbar(im[idx], cax=cbar_ax)
cbar_ax.tick_params(labelsize=cbarsize)

# Add Title Info
#fig.suptitle('Control Sorted by Mean Post-Blast', fontsize=14, fontweight='bold')

In [ ]:
laststds = ctlaftertraces[4,sortindsafter,:].std(axis=1)
firstmeans = ctlaftertraces[0,sortindsafter,:].mean(axis=1)
plt.plot(laststds)
plt.plot(firstmeans)
updiff = firstmeans >= 3*laststds #20 if mean, 152 if cell by cell
downdiff = -1*firstmeans >= 3*laststds #2 if mean, 8 if cell by cell
updiff.sum(), downdiff.sum(), firstmeans.shape, laststds.mean()

In [ ]:
#Plot Sets of Matches
upcells = ctlaftertraces[0,sortindsafter[updiff],:]
upbinary = ctlafterbinary[0,sortindsafter[updiff],:]
upstds = ctlaftertraces[4,sortindsafter[updiff],:]
upstdbinary = ctlafterbinary[4,sortindsafter[updiff],:]

downcells = ctlaftertraces[0,sortindsafter[downdiff],:]
downbinary = ctlafterbinary[0,sortindsafter[downdiff],:]
downstds = ctlaftertraces[4,sortindsafter[downdiff],:]
downstdbinary = ctlafterbinary[4,sortindsafter[downdiff],:]

labelsize = 14
axsize = 12
cbarsize = 12

fig, all_ax = plt.subplots(nrows=1,ncols=4, figsize=(15,5))

im=range(4)
for idx, ax in enumerate(all_ax):
    if idx==0:
        im[idx] = ax.imshow(upcells, cmap='seismic')
        ax.set_aspect(4)
        im[idx].set_clim(-50,50)
        ax.set_ylabel('Cell Number', fontsize=labelsize)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticks([0,50,100,150])
        ax.set_yticklabels([0,50,100,150], fontsize=axsize)
    elif idx==1: 
        im[idx] = ax.imshow(upstds, cmap='seismic')
        ax.set_aspect(4)
        im[idx].set_clim(-50,50)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticklabels([])
    elif idx==2: 
        im[idx] = ax.imshow(downcells, cmap='seismic')
        ax.set_aspect(16)
        im[idx].set_clim(-50,50)
        ax.set_ylabel('Cell Number', fontsize=labelsize)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticks([0,8])
        ax.set_yticklabels([0,8], fontsize=axsize)
    elif idx==3:
        im[idx] = ax.imshow(downstds, cmap='seismic')
        ax.set_aspect(16)
        im[idx].set_clim(-50,50)
        ax.set_xlabel('Time (sec)', fontsize=labelsize)
        ax.set_xticklabels(timelabels, fontsize=axsize)
        ax.set_yticklabels([])
        
# Add Colorbar
fig.subplots_adjust(right=0.95, wspace=0.3)
cbar_ax = fig.add_axes([0.97, 0.4, 0.02, 0.2])
fig.colorbar(im[idx], cax=cbar_ax, ticks=[-50,-25,0,25,50])
cbar_ax.tick_params(labelsize=cbarsize)

In [ ]:
plt.plot(upbinary[8])
plt.plot(upcells[8])
plt.figure()
plt.plot(upbinary.sum(axis=1))
print(upbinary.sum(axis=1).mean())

In [ ]:
#Calculate percentage cells for all
expcells = np.zeros((6,3))
ctlcells = np.zeros((6,3))
row = 0

for idx, cells in enumerate(np.array(([6,8],[9,10],[12,11]))):
    for ind in np.array([0,1]):
        #Pull out Mice/Traces
        afterexp = afterkeeptraces[list(celldict[cells[0]][1])[ind]]
        afterctl = afterkeeptraces[list(celldict[cells[1]][1])[ind]]
        #print(list(celldict[cells[0]][1])[ind])
        #print(list(celldict[cells[1]][1])[ind])
        #Calculate Significant Cells
        #Experimental
        expmeanfirstafter = afterexp[0,:,:].mean(axis=1)
        expsortindsafter = np.argsort(expmeanfirstafter)[::-1]
        explaststds = afterexp[4,expsortindsafter,:].std(axis=1)
        expfirstmeans = afterexp[0,expsortindsafter,:].mean(axis=1)
        expupdiff = expfirstmeans >= 3*explaststds
        expdowndiff = -1*expfirstmeans >= 3*explaststds
        #Control
        ctlmeanfirstafter = afterctl[0,:,:].mean(axis=1)
        ctlsortindsafter = np.argsort(ctlmeanfirstafter)[::-1]
        ctllaststds = afterctl[4,ctlsortindsafter,:].std(axis=1)
        ctlfirstmeans = afterctl[0,ctlsortindsafter,:].mean(axis=1)
        ctlupdiff = ctlfirstmeans >= 3*ctllaststds
        ctldowndiff = -1*ctlfirstmeans >= 3*ctllaststds
        #Fill Output Vectors (row/cell is updiff, downdiff, total)
        expcells[row,0] = expupdiff.sum()
        expcells[row,1] = expdowndiff.sum()
        expcells[row,2] = expfirstmeans.shape[0]
        ctlcells[row,0] = ctlupdiff.sum()
        ctlcells[row,1] = ctldowndiff.sum()
        ctlcells[row,2] = ctlfirstmeans.shape[0]
        row += 1

In [ ]:
expcells, ctlcells, expcells[:,2].sum(), ctlcells[:,2].sum()

In [ ]:
expupfrac = expcells[:,0] / expcells[:,2]
expdownfrac = expcells[:,1] / expcells[:,2]
ctlupfrac = ctlcells[:,0] / ctlcells[:,2]
ctldownfrac = ctlcells[:,1] / ctlcells[:,2]
np.sort(expupfrac), np.sort(expdownfrac), np.sort(ctlupfrac), np.sort(ctldownfrac)
[expupmean, expupstd] = [expupfrac.mean(), expupfrac.std()]
[expdownmean, expdownstd] = [expdownfrac.mean(), expdownfrac.std()]
[ctlupmean, ctlupstd] = [ctlupfrac.mean(), ctlupfrac.std()]
[ctldownmean, ctldownstd] = [ctldownfrac.mean(), ctldownfrac.std()]
print((expupmean, expupstd, np.mean(np.sum(expcells[:,0])/np.sum(expcells[:,2]))))
print((expdownmean, expdownstd, np.mean(np.sum(expcells[:,1])/np.sum(expcells[:,2]))))
print((ctlupmean, ctlupstd, np.mean(np.sum(ctlcells[:,0])/np.sum(ctlcells[:,2]))))
print((ctldownmean, ctldownstd, np.mean(np.sum(ctlcells[:,1])/np.sum(ctlcells[:,2]))))

In [ ]:
np.sum(ctlcells[:,0])

In [ ]:
(expupmed, ctlupmed) = (np.median(expupfrac),np.median(ctlupfrac))
(expdownmed, ctldownmed) = (np.median(expdownfrac),np.median(ctldownfrac))

#Mann Whitney U Test for Population Cells - Stats
(sstats.mannwhitneyu(expupfrac,ctlupfrac, alternative='two-sided'), expupmed, ctlupmed,
sstats.mannwhitneyu(expdownfrac,ctldownfrac, alternative='two-sided'), expdownmed, ctldownmed)

In [ ]:
#Mann Whitney U Test Plotting
inds = np.arange(2)
width = 0.4

fig, ax = plt.subplots()
expbars = ax.bar(inds, [expupfrac.mean(), expdownfrac.mean()], width, color='m', alpha=0.75,
                 yerr=[expupfrac.std(), expdownfrac.std()], error_kw=dict(ecolor='black', lw=1, capsize=3, capthick=1))

ctlbars = ax.bar(inds + width, [ctlupfrac.mean(), ctldownfrac.mean()], width, color='g', alpha = 0.75,
                 yerr=[ctlupfrac.std(), ctldownfrac.std()], error_kw=dict(ecolor='black', lw=1, capsize=3, capthick=1))

#ax.annotate('*', xy=(width-.01,0.3725), fontweight='bold')
#ax.annotate('', xy=(width/2,0.325), xytext=(width+width/2,0.325),
#            arrowprops={'connectionstyle':'bar', 'arrowstyle':'-', 'shrinkA':10, 'shrinkB':20,'lw':1})

ax.annotate('*', xy=(1+width-.01,0.235), fontweight='bold')
ax.annotate('', xy=(1+width/2,0.185), xytext=(1+width+width/2,0.185),
            arrowprops={'connectionstyle':'bar', 'arrowstyle':'-', 'shrinkA':10, 'shrinkB':20,'lw':1})

# add some text for labels, title and axes ticks
ax.set_ylabel('Fraction of Cells')
#ax.set_title('Testing Data')
ax.set_xticks(inds + width)
ax.set_xticklabels(('Sustained', 'Suppressed'))

ax.legend((expbars[0], ctlbars[0]), ('Blast', 'Sham'))

In [ ]:
outdir = '/mnt/BME-HAN-Storage/People/Hua-an/Data/TBI/Ali/'
outfn = 'Mouse11D1UpDownCells.mat'

sio.savemat(outdir+outfn, {'Mouse11D1Upinds': sortindsafter[updiff], 'Mouse11D1Downinds': sortindsafter[downdiff]})

In [ ]:
# Individual Cells
# Sorting
cell, day = 8, 1 #Be 0 or 1 for day 1 or 2
beforecell = celldict[cell][0][day]
aftercell = celldict[cell][1][day]
meanbefore = beforetraces[beforecell].std(axis=1)
sortindsbefore = np.argsort(meanbefore)[::-1]
meanfirstafter = aftertraces[aftercell][0,:,:].std(axis=1)
sortindsafter = np.argsort(meanfirstafter)[::-1]


plt.jet()
fig, all_ax = plt.subplots(nrows=1,ncols=6, figsize=(15,5))

im=range(6)
for idx, ax in enumerate(all_ax):
    if idx==0:
        im[idx] = ax.imshow(beforetraces[beforecell][sortindsbefore,:])
        ax.set_aspect('auto')
        #plt.colorbar(im[idx])
        im[idx].set_clim(-40,20)
        ax.set_ylabel('Cell #')
        ax.set_xlabel('Frame #')
    else:
        im[idx] = ax.imshow(aftertraces[aftercell][idx-1,sortindsafter,:])
        ax.set_aspect('auto')
        #plt.colorbar(im[idx])
        im[idx].set_clim(-40,20)
        ax.set_xlabel('Frame #')
    
plt.colorbar(im[5])

In [ ]:
# Sorting
meanfirstafter = allaftertraces[0,:,:].mean(axis=1)
sortindsafter = np.argsort(meanfirstafter)[::-1]
meanbefore = allbeforetraces.mean(axis=1)
sortindsbefore = np.argsort(meanbefore)[::-1]


plt.jet()
fig, all_ax = plt.subplots(nrows=1,ncols=6, figsize=(15,5))

im=range(6)
for idx, ax in enumerate(all_ax):
    if idx==0:
        im[idx] = ax.imshow(allbeforetraces[sortindsbefore,:])
        plt.colorbar(im[idx])
        im[idx].set_clim(-40,20)
        ax.set_ylabel('Cell #')
        ax.set_xlabel('Frame #')
    else:
        im[idx] = ax.imshow(allaftertraces[idx-1,sortindsafter,:])
        #plt.colorbar(im[idx])
        im[idx].set_clim(-40,20)
        ax.set_xlabel('Frame #')
    
#plt.colorbar(im[5])

In [ ]:
plt.plot(allaftertraces[0,10,:])
np.min(allbeforetraces), np.max(allbeforetraces)